In [35]:
from tqdm import tqdm
from dart_fss import get_corp_list
from dotenv import load_dotenv
from sklearn.preprocessing import StandardScaler
import dart_fss as dart
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import requests
import json
import os
import time

load_dotenv()

API_KEY = os.getenv("API_KEY")

dart.set_api_key(api_key=API_KEY)

# 모든 상장된 기업 리스트 불러오기
corp_list = get_corp_list()


# 삼성전자를 이름으로 찾기 ( 리스트 반환 )
samsung = corp_list.find_by_corp_name('삼성전자', exactly=True)[0]
samsung

Label,Data
corp_code,00126380
corp_name,삼성전자
corp_eng_name,"SAMSUNG ELECTRONICS CO,.LTD"
stock_code,005930
modify_date,20250326
sector,통신 및 방송 장비 제조업
product,"통신 및 방송 장비 제조(무선) 제품, 반도체 제조(메모리) 제품, 전자부품 제조(디스플레이) 제품, 영상 및 음향기기 제조(영상기기) 제품 등"
corp_cls,Y


In [20]:
industry = "도매및소매업" 

df_kospi = pd.read_excel(f"업종별_기업리스트/{industry}_KOSPI.xlsx")
df_kosdaq = pd.read_excel(f"업종별_기업리스트/{industry}_KOSDAQ.xlsx")

df_total = pd.concat([df_kospi, df_kosdaq], ignore_index=True)
df_total = df_total.drop_duplicates()
df_total["업종구분"] = industry

corp_name_list = list(df_total["회사명"])
print(f"{industry} {len(corp_name_list)}개 기업 목록을 리스트로 저장했습니다.")
# corp_name_list

도매및소매업 172개 기업 목록을 리스트로 저장했습니다.


c:\Users\82107\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\82107\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [21]:
target_ratio = [
    "자기자본비율", "부채비율", "유동비율", "유동부채비율", "비유동부채비율", "비유동비율", "금융비용부담률",
    "자본유보율", "재무레버리지", "비유동적합률", "비유동자산구성비율", "유형자산구성비율", "유동자산구성비율", 
    "재고자산구성비율", "유동자산/비유동자산비율", "재고자산/유동자산비율", # 안정성지표
]

idx_cl_code_dict = {
    # "M210000": "수익성지표",
    "M220000": "안정성지표",
    # "M230000": "성장성지표",
    # "M240000": "활동성지표"
}

corp_name_dict = {}

for corp in corp_name_list:
    name = corp_list.find_by_corp_name(corp, exactly=True)
    if name:
        corp_code = str(name[0])[1:9]
    else:
        print(f"{corp}(은)는 다중회사가 아니므로 건너뜁니다.")
        continue
    corp_name_dict[f"{corp}"] = corp_code
    # print(f"{corp}의 회사 이름과 고유번호를 딕셔너리에 추가했습니다.")

print(f"회사는 총 {len(corp_name_dict)}개입니다.")
# print(f"회사명&고유번호 딕셔너리:\n{corp_name_dict}")

# 기업별 종합 row
corp_data_dict = {}

회사는 총 172개입니다.


In [24]:
# --- API 요청 URL 및 파라미터 ---
for idx_cl_code, 지표분류 in idx_cl_code_dict.items():

    tqdm.write(f"=== {지표분류} 요청 중 ===")

    for corp_name, corp_code in tqdm(corp_name_dict.items(), desc=f"{지표분류} 수집 중", unit="개"):
        base_url = "https://opendart.fss.or.kr/api/fnlttCmpnyIndx.json"
        params = {
            'crtfc_key': API_KEY,
            'corp_code': corp_code, # 대상 기업 코드
            'bsns_year': "2024", # 회계연도
            'reprt_code': "11011", # 사업보고서
            'idx_cl_code': idx_cl_code,
        }

        # --- GET 요청 보내기 ---
        try:
            response = requests.get(base_url, params=params)
            
            # --- 응답 확인 ---
            # HTTP 상태 코드가 200 (OK)인지 확인
            if response.status_code == 200:
                # print(response.text) 분석&코드 작성 시 사용
                
                data = json.loads(response.text)

                # 한 기업 당 하나의 행 생성
                if corp_name not in corp_data_dict:
                    corp_data_dict[corp_name] = {
                        "기업명": corp_name,
                    }

                for item in data.get("list", []):
                    idx_nm = item.get("idx_nm")
                    if idx_nm in target_ratio:
                        corp_data_dict[corp_name][idx_nm] = item.get("idx_val")
                
            else:
                tqdm.write(f"HTTP 오류 {response.status_code} - {corp_name}")

            time.sleep(0.3)

        except requests.exceptions.RequestException as e:
            tqdm.write(f"{corp_name} 요청 중 에러 발생: {e}")

=== 안정성지표 요청 중 ===


안정성지표 수집 중: 100%|██████████| 172/172 [01:08<00:00,  2.51개/s]


In [25]:
# DataFrame 생성
df = pd.DataFrame(list(corp_data_dict.values()))

# CSV 저장
df.to_csv("주요지표.csv", index=False, encoding='utf-8-sig')

In [39]:
df = pd.read_csv("주요지표.csv")

filtered_df = df.dropna(subset=target_ratio, how='all')

print(f"원본 행 개수: {len(df)}")
print(f"필터링 후 행 개수: {len(filtered_df)}")

filtered_df.to_csv("주요지표_누락처리.csv", index=False, encoding='utf-8-sig')

원본 행 개수: 172
필터링 후 행 개수: 149


In [ ]:
z_df = filtered_df.copy()

for col in target_ratio:
    z_df[col] = (
        z_df[col]
        .astype(str)
        .str.replace(",", "")   # 쉼표 제거
        .str.replace("%", "")   # 퍼센트 제거
        .replace("None", np.nan)
        .replace("", np.nan)
        .astype(float)
    )

mean_val = z_df["자본유보율"].mean()
print(f"자본유보율 평균: {mean_val}")

# GS리테일의 값 확인
gs_val = z_df.loc[z_df["기업명"] == "GS리테일", "자본유보율"].values[0]
print(f"GS리테일 자본유보율: {gs_val}")

# 수식으로 Z-score 직접 계산
z_score = (gs_val - mean_val) / z_df["자본유보율"].std()
print(f"Z-score (수식 기반): {z_score}")

print(z_df.dtypes)

자본유보율 평균: 1554.9254038461538
GS리테일 자본유보율: 4417.948
Z-score (수식 기반): 1.4584973944221031
기업명              object
자기자본비율          float64
부채비율            float64
유동비율            float64
유동부채비율          float64
비유동부채비율         float64
비유동비율           float64
금융비용부담률         float64
자본유보율           float64
재무레버리지          float64
비유동적합률          float64
비유동자산구성비율       float64
유형자산구성비율        float64
유동자산구성비율        float64
재고자산구성비율        float64
유동자산/비유동자산비율    float64
재고자산/유동자산비율     float64
dtype: object


In [88]:
scaler = StandardScaler()
z_scaled = scaler.fit_transform(z_df[target_ratio])

# 다시 DataFrame으로 변환
z_scaled_df = pd.DataFrame(z_scaled, columns=target_ratio, index=z_df.index)
z_scaled_df = z_scaled_df.fillna(0)
z_scaled_df["기업명"] = filtered_df.loc[z_df.index, "기업명"].values
z_scaled_df.to_csv("주요지표_Z.csv", index=False, encoding='utf-8-sig')